In [1]:
def compute_conv_output_size(
        input_size,   # (H, W)
        in_channels,
        out_channels,
        kernel_size,
        stride=1,
        padding=0,
        dilation=1
    ):
    """
    Computes output (H, W), output channels, and total features after a Conv2D-like layer.
    Supports int or tuple arguments.
    """
    # unpack input H, W
    H, W = input_size

    # allow int or tuple kernel/stride/paddisng/dilation
    def to_tuple(x):
        return (x, x) if isinstance(x, int) else x

    KH, KW = to_tuple(kernel_size)
    SH, SW = to_tuple(stride)
    PH, PW = to_tuple(padding)
    DH, DW = to_tuple(dilation)

    # compute output height and width using PyTorch formula
    out_H = ((H + 2*PH - DH*(KH - 1) - 1) // SH) + 1
    out_W = ((W + 2*PW - DW*(KW - 1) - 1) // SW) + 1

    # output channels always = out_channels
    out_C = out_channels

    # total number of features
    total_features = out_C * out_H * out_W

    return (out_C, out_H, out_W, total_features)


In [6]:
print(compute_conv_output_size(
    input_size=(32, 32),
    in_channels=3,
    out_channels=10,
    kernel_size=3,
    stride=1,
    padding=0
))

(10, 30, 30, 9000)


# Loading Pretrained Models

In [ ]:
import torch

def load_pretrained_weights(model, state_dict_path=None, pretrained_model=None, strict=False):
    """
    Load pretrained weights into a flexible model, skipping layers that don't match in shape.

    Args:
        model: nn.Module - your flexible model
        state_dict_path: str - path to pretrained state_dict (optional)
        pretrained_model: nn.Module - another model with pretrained weights (optional)
        strict: bool - whether to enforce exact match for remaining layers (default False)
    """

    if state_dict_path:
        # Load weights from a saved checkpoint
        pretrained_dict = torch.load(state_dict_path, map_location='cpu')
        if 'state_dict' in pretrained_dict:
            pretrained_dict = pretrained_dict['state_dict']
    elif pretrained_model:
        # Get state_dict from another model
        pretrained_dict = pretrained_model.state_dict()
    else:
        raise ValueError("Either state_dict_path or pretrained_model must be provided.")

    model_dict = model.state_dict()

    # Filter out layers that don't match in shape
    filtered_dict = {}
    for k, v in pretrained_dict.items():
        if k in model_dict:
            if model_dict[k].shape == v.shape:
                filtered_dict[k] = v
            else:
                print(f"Skipping layer {k}: size mismatch {v.shape} vs {model_dict[k].shape}")
        else:
            print(f"Skipping layer {k}: not found in target model")

    # Load matched layers
    model_dict.update(filtered_dict)
    model.load_state_dict(model_dict, strict=strict)
    print(f"Loaded {len(filtered_dict)} / {len(model_dict)} layers from pretrained weights.")

    return model


In [ ]:
import torchvision.models as models

# Suppose we want to use AlexNet pretrained weights
pretrained_alexnet = models.alexnet(pretrained=True)

# Our flexible AlexNet with different input channels / classes
alex_cnn = AlexNet(in_channels=1, num_classes=10)

# Load pretrained weights where shapes match
alex_cnn = load_pretrained_weights(alex_cnn, pretrained_model=pretrained_alexnet)


In [ ]:
# Path to checkpoint
checkpoint_path = "alexnet_cifar10.pth"
alex_cnn = load_pretrained_weights(alex_cnn, state_dict_path=checkpoint_path)


### Calculate the output size of 1D convolution 

In [1]:
def conv_output_size(input_size, kernel_size, stride=1, padding=0):
    """
    Calculate the output size of a 1D convolution operation.

    Formula:
        output = floor((W - K + 2P) / S) + 1

    Parameters
    ----------
    input_size : int
        The size of the input (W).
    kernel_size : int
        The size of the convolution kernel/filter (K).
    stride : int, optional (default=1)
        Step size with which the filter moves across the input (S).
    padding : int, optional (default=0)
        The number of padded zeros added to both sides of the input (P).

    Returns
    -------
    int
        The computed output size after applying convolution.

    Raises
    ------
    ValueError
        If kernel_size is larger than the padded input size.

    Examples
    --------
    >>> conv_output_size(32, 3, stride=1, padding=1)
    32

    >>> conv_output_size(28, 5, stride=2, padding=0)
    12
    """
    # Calculate effective input size after padding
    effective_input = input_size + 2 * padding

    if kernel_size > effective_input:
        raise ValueError("Kernel size cannot be larger than padded input size.")

    # Apply convolution formula
    output = (effective_input - kernel_size) // stride + 1

    return output


In [2]:
print(conv_output_size(32, 3, stride=1, padding=1))   # 32

32


### Calculate the convolutin Output with no padding or stride

In [4]:
def conv_output_no_padding_no_stride(input_size, kernel_size):
    """
    Compute the output size of a 1D convolution when:
      - No padding (P = 0)
      - No stride (S = 1)

    Formula:
        Output = W - K + 1

    Parameters
    ----------
    input_size : int
        Size of the input (W).
    kernel_size : int
        Size of the convolution kernel/filter (K).

    Returns
    -------
    int
        The output size after applying convolution.

    Raises
    ------
    ValueError
        If kernel_size is larger than the input_size.

    Examples
    --------
    >>> conv_output_no_padding_no_stride(10, 3)
    8
    """
    if kernel_size > input_size:
        raise ValueError("Kernel size cannot be larger than input size.")

    return input_size - kernel_size + 1


### Padding for same size as input

In [6]:
def same_padding(kernel_size):
    """
    Compute the required padding (P) for a convolution to produce
    the same output size as the input (“same” convolution).

    Formula:
        P = (K - 1) / 2

    Conditions:
        - Works only when kernel_size (K) is odd.
        - Stride must be 1 for exact same-size output.

    Parameters
    ----------
    kernel_size : int
        Size of the kernel/filter (K). Must be an odd integer.

    Returns
    -------
    int
        The required padding amount P.

    Raises
    ------
    ValueError
        If kernel_size is not odd.

    Examples
    --------
    >>> same_padding(3)
    1

    >>> same_padding(5)
    2
    """
    if kernel_size % 2 == 0:
        raise ValueError("Kernel size must be odd to achieve same output size.")

    return (kernel_size - 1) // 2


### Height and Width of 2D pooling layer

In [7]:
def pooling_output_size(input_height, input_width, kernel_size, stride):
    """
    Compute the output height and width of a 2D pooling layer.

    Formula:
        H' = (H - K) / S + 1
        W' = (W - K) / S + 1

    Assumes:
        - No padding (typical for pooling layers)
        - Kernel size and stride are integers
        - (H - K) and (W - K) must be divisible by S for valid output

    Parameters
    ----------
    input_height : int
        Height of the input feature map (H).
    input_width : int
        Width of the input feature map (W).
    kernel_size : int
        Pooling kernel size (K).
    stride : int
        Stride of the pooling operation (S).

    Returns
    -------
    (int, int)
        Output height H' and width W'.

    Raises
    ------
    ValueError
        If kernel size is larger than input dimensions,
        or if the dimensions are not divisible by stride.

    Examples
    --------
    >>> pooling_output_size(28, 28, kernel_size=2, stride=2)
    (14, 14)

    >>> pooling_output_size(32, 32, kernel_size=4, stride=2)
    (15, 15)
    """
    if kernel_size > input_height or kernel_size > input_width:
        raise ValueError("Kernel size cannot be larger than input dimensions.")

    if (input_height - kernel_size) % stride != 0:
        raise ValueError("Input height is not compatible with stride.")
    if (input_width - kernel_size) % stride != 0:
        raise ValueError("Input width is not compatible with stride.")

    out_height = (input_height - kernel_size) // stride + 1
    out_width = (input_width - kernel_size) // stride + 1

    return out_height, out_width


### Receptive Field

In [9]:
def receptive_field(kernel_size, num_layers):
    """
    Compute the receptive field (RF) size of a stack of convolutional layers.

    Formula:
        RF = 1 + L * (K - 1)

    Assumes:
        - Stride = 1 for all layers
        - All layers have the same kernel size

    Parameters
    ----------
    kernel_size : int
        Size of the convolution kernel/filter (K).
    num_layers : int
        Number of successive convolutional layers (L).

    Returns
    -------
    int
        The receptive field size of the network.

    Examples
    --------
    >>> receptive_field(3, 1)
    3

    >>> receptive_field(3, 4)
    10
    """
    if kernel_size < 1:
        raise ValueError("Kernel size must be at least 1.")
    if num_layers < 1:
        raise ValueError("Number of layers must be at least 1.")

    return 1 + num_layers * (kernel_size - 1)


### Resize Transforms

In [2]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Upscale to AlexNet's expected size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])